### **MODEL TRAINING**

#### 1.1 *Import Libraries*

In [ ]:
### Basic Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Modelling 
from sklearn.metrics import r2_score,mean_absolute_error,root_mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.svm import SVR
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
import warnings

#### *Import CSV as Pandas DataFrame*

In [ ]:
df=pd.read_csv("../data/Cleaned_Carbon_Emission.csv")
df.head()

### Prepare X and y Variable


In [ ]:
X=df.drop(columns=['CarbonEmission'],axis=1)
X

In [ ]:
y=df['CarbonEmission']
y

In [ ]:
# Create Column Transformer with 3 types of transformers
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [ ]:
X=preprocessor.fit_transform(X)

Splitting The Dataset Into Training Data And Testing Data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

Create an Evaluate Function to give all metrics after model Training


In [ ]:
def evaluate_model(true,predicted):
    RMSE=root_mean_squared_error(y_true=true,y_pred=predicted)
    MAE=mean_absolute_error(y_true=true,y_pred=predicted)
    R2=r2_score(y_true=true,y_pred=predicted)
    return MAE,RMSE,R2

In [ ]:
models= {
    "Linear Regression": LinearRegression(),
    "Lasso Regression": Lasso(),
    "Ridge Regression": Ridge(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest": RandomForestRegressor(),
    "K Nearest Neighbors": KNeighborsRegressor(),
    "Xgboost": XGBRegressor(),
    "AdaBoost": AdaBoostRegressor()
}

model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train,y_train) # Train model

    ##model prediction
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    # Evaluate Train and Test dataset
    model_train_mae , model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)

    model_test_mae , model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)

    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    
    print('Model performance for Training set')
    print("- Root Mean Squared Error: {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_train_mae))
    print("- R2 Score: {:.4f}".format(model_train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("- Root Mean Squared Error: {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error: {:.4f}".format(model_test_mae))
    print("- R2 Score: {:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')

In [ ]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

### Interpretation & Insights
#### Linear, Lasso, Ridge
- Similar performance indicates your data has linear relationships.
- Great baseline models.
- Lasso slightly improved test performance → suggests feature selection matters.

#### Decision Tree
- R² = 1.0 on training, but low on test → classic overfitting.
- Tree memorized data instead of learning patterns.

#### Random Forest
- Less overfitting than Decision Tree.
- High training score but good generalization.
- Good balance, could be improved with tuning.

#### KNN
Performs poorly → dataset probably:
- has irrelevant distance patterns
- may not be normalized properly
- not suited for simple instance-based learning

#### XGBoost
- Best model: R² = 0.97, low RMSE and MAE.
- Handles feature interactions, non-linear patterns, outliers well.
- Excellent generalization, no heavy overfitting.

#### Saving the best model

In [ ]:
model=XGBRegressor()
model.fit(X_train,y_train)
y_test_pred=model.predict(X_test)
evaluate_model(y_test,y_test_pred)

In [ ]:
import joblib

joblib.dump(model,"artifacts/model.pkl")

print("Model saved successfully!")